In [59]:
import os

import pandas as pd

DATA_PATH = '/opt/ml/input/cropped_v2.1/train/'
IMG_PATH = '/opt/ml/input/cropped_v2.1/train/images/'

In [61]:
df = pd.read_csv('/opt/ml/input/cropped_v2.1/train/train_outlier.csv')
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [62]:
img_path_list = []

In [63]:
for p in df['path'].values:
    try : img_list = sorted([im for im in os.listdir(IMG_PATH+p) if '._' not in im])
    except : pass
    for im in img_list:
        img_path_list.append(p+'/'+im)

In [64]:
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [65]:
len(img_path_list)

18900

In [66]:
img_path_list[0:10]

['000001_female_Asian_45/incorrect_mask.jpg',
 '000001_female_Asian_45/mask1.jpg',
 '000001_female_Asian_45/mask2.jpg',
 '000001_female_Asian_45/mask3.jpg',
 '000001_female_Asian_45/mask4.jpg',
 '000001_female_Asian_45/mask5.jpg',
 '000001_female_Asian_45/normal.jpg',
 '000002_female_Asian_52/incorrect_mask.jpg',
 '000002_female_Asian_52/mask1.jpg',
 '000002_female_Asian_52/mask2.jpg']

In [67]:
df_cls = pd.DataFrame( { 'img_path' : img_path_list} )
df_cls

,img_path
0,000001_female_Asian_45/incorrect_mask.jpg
1,000001_female_Asian_45/mask1.jpg
2,000001_female_Asian_45/mask2.jpg
3,000001_female_Asian_45/mask3.jpg
4,000001_female_Asian_45/mask4.jpg
...,...
18895,006959_male_Asian_19/mask2.jpg
18896,006959_male_Asian_19/mask3.jpg
18897,006959_male_Asian_19/mask4.jpg
18898,006959_male_Asian_19/mask5.jpg


In [68]:
def path_finder(x):
    return x.split('/')[0]

In [69]:
def name_finder(x):
    return x.split('/')[-1]

In [70]:
def mask_classifier(x):
    mask = x.split('.')[0]
    
    if mask == 'incorrect_mask':
        return 'Incorrect'
    elif mask == 'normal':
        return 'Not Wear'
    else:
        return 'Wear'

In [71]:
df_cls['path'] = df_cls['img_path'].map(path_finder)
df_cls['name'] = df_cls['img_path'].map(name_finder)
df_cls['mask'] = df_cls['name'].map(mask_classifier)

In [72]:
df_cls = pd.merge(df, df_cls, on='path')
# df_cls = pd.merge(df, df_cls, on='img_path')
df_cls

,id,gender,race,age,path,img_path,name,mask
0,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/incorrect_mask.jpg,incorrect_mask.jpg,Incorrect
1,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask1.jpg,mask1.jpg,Wear
2,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask2.jpg,mask2.jpg,Wear
3,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask3.jpg,mask3.jpg,Wear
4,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask4.jpg,mask4.jpg,Wear
...,...,...,...,...,...,...,...,...
18895,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask2.jpg,mask2.jpg,Wear
18896,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask3.jpg,mask3.jpg,Wear
18897,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask4.jpg,mask4.jpg,Wear
18898,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask5.jpg,mask5.jpg,Wear


In [73]:
df_cls['target'] = 0


df_cls['target'] = ((df_cls['mask']=='Not Wear')*12 + 
                  (df_cls['mask']=='Incorrect')*6 +
                  (df_cls['gender']=='female')*3 + 
                  (df_cls['age']>=30)*1 + 
                  (df_cls['age']>=60)*1)


In [74]:
df_cls

,id,gender,race,age,path,img_path,name,mask,target
0,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/incorrect_mask.jpg,incorrect_mask.jpg,Incorrect,10
1,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask1.jpg,mask1.jpg,Wear,4
2,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask2.jpg,mask2.jpg,Wear,4
3,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask3.jpg,mask3.jpg,Wear,4
4,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask4.jpg,mask4.jpg,Wear,4
...,...,...,...,...,...,...,...,...,...
18895,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask2.jpg,mask2.jpg,Wear,0
18896,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask3.jpg,mask3.jpg,Wear,0
18897,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask4.jpg,mask4.jpg,Wear,0
18898,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask5.jpg,mask5.jpg,Wear,0


In [75]:
df_cls.to_csv(DATA_PATH+'train_classified.csv', index=False)

In [76]:
df_cls['target'] = 0
df_cls['target'] = ((df_cls['mask']=='Not Wear')*12 + 
                  (df_cls['mask']=='Incorrect')*6)
df_cls.to_csv(DATA_PATH+'train_classified_mask.csv', index=False)

In [77]:
df_cls['target'] = 0
df_cls['target'] = (df_cls['gender']=='female')*3
df_cls.to_csv(DATA_PATH+'train_classified_gender.csv', index=False)

In [78]:
df_cls['target'] = 0
df_cls['target'] = ((df_cls['age']>=30)*1 + 
                  (df_cls['age']>=60)*1)
df_cls.to_csv(DATA_PATH+'train_classified_age.csv', index=False)

In [79]:
df_cls

,id,gender,race,age,path,img_path,name,mask,target
0,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/incorrect_mask.jpg,incorrect_mask.jpg,Incorrect,1
1,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask1.jpg,mask1.jpg,Wear,1
2,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask2.jpg,mask2.jpg,Wear,1
3,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask3.jpg,mask3.jpg,Wear,1
4,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask4.jpg,mask4.jpg,Wear,1
...,...,...,...,...,...,...,...,...,...
18895,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask2.jpg,mask2.jpg,Wear,0
18896,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask3.jpg,mask3.jpg,Wear,0
18897,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask4.jpg,mask4.jpg,Wear,0
18898,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask5.jpg,mask5.jpg,Wear,0
